In [1]:
import pandas as pd
import sys

sys.path.append("../utils/")
from myFunctions import jsonGzipToDataframe2, toDommyColumns

Guardemos el JSON comprimido dentro de un dataframe Pandas

In [2]:
df = jsonGzipToDataframe2('../datasource/user_reviews.json.gz')

In [3]:
df.head(2)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


In [4]:
df.shape

(25799, 3)

Vamos a darnos una idea de como son los diccionarios en la columna 'reviews'

In [5]:
#Ver todos los diccionarios de la columna 'reviews' en la fila 2
#df['reviews'][0]
for dic in df.loc[2, 'reviews']:
    print(dic) # con dic['item_id'] veo solo los identificadores unicos de los juegos

{'funny': '', 'posted': 'Posted February 3.', 'last_edited': '', 'item_id': '248820', 'helpful': 'No ratings yet', 'recommend': True, 'review': "A suitably punishing roguelike platformer.  Winning feels good.  Progressive unlocks mean a good slog ending in failure doesn't feel like a waste."}
{'funny': '', 'posted': 'Posted December 4, 2015.', 'last_edited': 'Last edited December 5, 2015.', 'item_id': '370360', 'helpful': 'No ratings yet', 'recommend': True, 'review': '"Run for fun? What the hell kind of fun is that?"'}
{'funny': '', 'posted': 'Posted November 3, 2014.', 'last_edited': '', 'item_id': '237930', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Elegant integration of gameplay, story, world development and aesthetic.'}
{'funny': '', 'posted': 'Posted October 15, 2014.', 'last_edited': '', 'item_id': '263360', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Random drops and random quests, with stat points.  Animation style reminiscent of the era before 

Convirtamos los diccionarios que estan en 'reviews' en columnas

In [6]:
# Entre otras cosas, json_normalize corrige las comillas simples y dobles de JSON en 'reviews'
df = pd.json_normalize(df.to_dict('records'), 'reviews', ['user_id'])

# Reorganizar las columnas:
df = df[[
    'user_id',
    'item_id',
    'posted',
    'last_edited',
    'recommend',
    'funny',
    'helpful',
    'review'
]]

A ver cuantos NaN hay en cada columna

In [7]:
df.isna().sum() # le aplicamos una mascara al dataframe para sumar cuantos NaN hay 

user_id        0
item_id        0
posted         0
last_edited    0
recommend      0
funny          0
helpful        0
review         0
dtype: int64

In [8]:
df['item_id'] = df['item_id'].astype('int64')  # Convierte a entero de 64 bits

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   item_id      59305 non-null  int64 
 2   posted       59305 non-null  object
 3   last_edited  59305 non-null  object
 4   recommend    59305 non-null  bool  
 5   funny        59305 non-null  object
 6   helpful      59305 non-null  object
 7   review       59305 non-null  object
dtypes: bool(1), int64(1), object(6)
memory usage: 3.2+ MB


Listo los datos (masticados) para que los cinetificos de datos hagan lo suyo

In [10]:
# Guardar el DataFrame en un archivo .csv comprimido con gzip
df.to_csv('../datasource/user_reviews_chewed.csv.gz', compression = 'gzip', index = False)